In [4]:
# If you want to pip install inline run the following:
!pip install pandas
!pip install pandas_datareader
!pip install mpl_finance
!pip install mplfinance
!pip install --upgrade mplfinance

# Import Packages
import bs4 as bs
import pickle
import requests
import datetime as dt
import os

import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web
#import pandas.plotting
from pandas.plotting import register_matplotlib_converters
#import pandas.testing
from pandas.testing import assert_frame_equal
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
from matplotlib import style

import mplfinance  
from mpl_finance import candlestick_ohlc # mpl_finance is deprecated - need to update visualizations  
from concurrent import futures

import collections
from collections import Counter

import sklearn
from sklearn import svm, neighbors
from sklearn.svm import LinearSVC 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier

style.use('ggplot')

In [5]:
# Parts 1-3: Using Stock Market Data
# This example uses Tesla ($TSLA) stock data from 1/1/2000 to current date.
# df = pd.read_csv('c:/Users/JCW/Desktop/Stock_Market_Data_Analysis/Company Data/tsla.csv', parse_dates = True, index_col = 'Date')
start = dt.datetime(2000,1,1)
end = dt.datetime.now()
df = web.DataReader('TSLA', 'yahoo', start, end)

print(df.tail(10))

TypeError: string indices must be integers, not 'str'

In [ ]:
# Read data into csv.
df = web.DataReader('TSLA', 'yahoo', start, end)
df.to_csv('tsla.csv')
df = pd.read_csv('tsla.csv', parse_dates = True, index_col = 'Date')

# Moving average over 100 day period. You can alter the window to average over 30 days, or another time period.
df['100ma'] = df['Adj Close'].rolling(window = 100, min_periods = 0).mean()
df.dropna(inplace = True)
print(df.tail())

In [ ]:
# Plot data using two plots (one for adjusted close price, one for volume).
ax1 = plt.subplot2grid((6,1), (0,0), rowspan = 5, colspan = 1)
ax2 = plt.subplot2grid((6,1), (5,0), rowspan = 1, colspan = 1, sharex = ax1)

# Volume is shown in the subplot as a bar chart, and adj close is a line graph. Red is actual
# price, and blue is the 100 moving average.
ax1.plot(df.index, df['Adj Close'])
ax1.plot(df.index, df['100ma'])
ax2.bar(df.index, df['Volume'])
plt.show()

In [ ]:
df_ohlc = df['Adj Close'].resample('10D').ohlc()
df_volume = df['Volume'].resample('10D').sum()
df_ohlc.reset_index(inplace = True)
print(df_ohlc.head())

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
df_ohlc['Date'] = df_ohlc['Date'].map(mdates.date2num)
print(df_ohlc.head())

In [ ]:
ax1 = plt.subplot2grid((6,1), (0,0), rowspan = 5, colspan = 1)
ax2 = plt.subplot2grid((6,1), (5,0), rowspan = 1, colspan = 1, sharex = ax1)

candlestick_ohlc(ax1, df_ohlc.values, width = 2, colorup = 'g')
ax2.fill_between(df_volume.index.map(mdates.date2num), df_volume.values, 0)
plt.show()

In [ ]:
# Part 4: Resampling
# Here we are using a resampling method for the data sample to improve accuracy 
df = pd.read_csv('tsla.csv', parse_dates = True, index_col = 0)
df['100ma'] = df['Adj Close'].rolling(window = 100, min_periods = 0).mean()
print(df.tail(10))

df_ohlc = df['Adj Close'].resample('10D').mean()

# Resample data for 10 day period
df_ohlc = df['Adj Close'].resample('10D').ohlc()
df_volume = df['Volume'].resample('10D').sum()
df_ohlc.reset_index(inplace = True)

# Convert datetime object to mdate
df_ohlc['Date'] = df_ohlc['Date'].map(mdates.date2num)
print(df_ohlc.head())

ax1 = plt.subplot2grid((6,1), (0,0), rowspan = 5, colspan = 1)
ax2 = plt.subplot2grid((6,1), (5,0), rowspan = 1, colspan = 1, sharex = ax1)
ax1.xaxis_date()

candlestick_ohlc(ax1, df_ohlc.values, width = 3, colorup = 'g')
ax2.fill_between(df_volume.index.map(mdates.date2num), df_volume.values, 0)
                 
ax1.plot(df.index, df['Adj Close'])
ax1.plot(df.index, df['100ma'])
ax2.bar(df.index, df['Volume'])

plt.show()

In [ ]:
# Part 5: Getting S&P500 List
# Here we are using BeautifulSoup to scrape data from wikipedia, and save the current S&P500 list.
def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        tickers.append(ticker)
        
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)

    print(tickers)

    return tickers

save_sp500_tickers()

In [ ]:
# Part 6: Getting Stock Prices
# Get data from Yahoo and call SP500 tickers list as sp500
def get_data_from_yahoo(reload_sp500 = False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)

# Take all of the data for stocks and store in a directory
# Working with API, parsing website, take entire dataset and store locally
# Here we will look at Adjusted Close, but we can look at other columns later
    if not os.path.exists('stock_dfs2'):
        os.makedirs('stock_dfs2')
    
    start = dt.datetime(2016,1,1)        
    end = dt.datetime.now()
    #end = dt.datetime(2020,7,24)

    # Grab all ticker data
    for ticker in tickers:
        print(ticker)    
        if not os.path.exists('stock_dfs2/{}.csv'.format(ticker)):
            try:
                df = web.DataReader(ticker, 'yahoo', start, end)
                df.to_csv('stock_dfs2/{}.csv'.format(ticker))
            except:
                print(f'Problems retrieving data for {ticker}. Skipping!')
            else:
                print('Already have {}'.format(ticker))

get_data_from_yahoo()

In [ ]:
# Part 7: Combine S&P500 List with Adjusted Close Price for All Companies
def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)
    mainDataSet = pd.DataFrame()

# Count in SP500 tickers list
    for count, ticker in enumerate(tickers):
        fileDataSet = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        fileDataSet.set_index('Date', inplace = True)
        fileDataSet.rename(columns = {'Adj Close':ticker}, inplace = True)
        fileDataSet.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace = True)

        if mainDataSet.empty:
            mainDataSet = fileDataSet
        else:
            mainDataSet = mainDataSet.join(fileDataSet)
        print(mainDataSet.head())
    mainDataSet.to_csv('sp500_joined_closes.csv')
compile_data()  

                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...        APD     AKAM       ALK  \
Date                                ...                                 
2000-01-03  15.500  NaN  28.294199  ...  18.104057  321.250  7.803120   
2000-01-04  14.625  NaN  27.172190  ...  17.552309  300.000  7.705581   
2000-01-05  15.000  NaN  27.464891  ...  17.655748  283.500  7.747382   
2000-01-06  16.000  NaN  27.684412  ...  18.828213  236.125  7.754352   
2000-01-07  16.250  Na

                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...  ALLE       LNT        ALL  GOOGL  \
Date                                ...                                     
2000-01-03  15.500  NaN  28.294199  ...   NaN  5.336293  14.409927    NaN   
2000-01-04  14.625  NaN  27.172190  ...   NaN  5.336293  13.950034    NaN   
2000-01-05  15.000  NaN  27.464891  ...   NaN  5.597509  14.563220    NaN   
2000-01-06  16.000  NaN  27.684412  ...   NaN  5.597509  14.486568    NaN  

                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...     AMZN  AMCR        AEE  AAL  \
Date                                ...                                  
2000-01-03  15.500  NaN  28.294199  ...  89.3750   NaN  11.942023  NaN   
2000-01-04  14.625  NaN  27.172190  ...  81.9375   NaN  11.942023  NaN   
2000-01-05  15.000  NaN  27.464891  ...  69.7500   NaN  12.403996  NaN   
2000-01-06  16.000  NaN  27.684412  ...  65.5625   NaN  12.357801  NaN   
2000-01-07  16.2

[5 rows x 40 columns]
                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...          AIG        AMT  AWK  AMP  \
Date                                ...                                     
2000-01-03  15.500  NaN  28.294199  ...  1002.118225  25.186090  NaN  NaN   
2000-01-04  14.625  NaN  27.172190  ...   950.897522  24.917585  NaN  NaN   
2000-01-05  15.000  NaN  27.464891  ...   953.157166  25.669407  NaN  NaN   
2000-01-06  16.000  NaN  27.684412  ...   982.231995 

                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...      ANSS  ANTM        AON       AOS  \
Date                                ...                                        
2000-01-03  15.500  NaN  28.294199  ...  2.765625   NaN  27.607428  0.643602   
2000-01-04  14.625  NaN  27.172190  ...  2.687500   NaN  26.988020  0.634328   
2000-01-05  15.000  NaN  27.464891  ...  2.703125   NaN  27.784399  0.632473   
2000-01-06  16.000  NaN  27.684412  ...  2.703125   NaN  28.

...

                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...        VRSN  VRSK         VZ  \
Date                                ...                                
2000-01-03  15.500  NaN  28.294199  ...  161.444855   NaN  21.479733   
2000-01-04  14.625  NaN  27.172190  ...  159.958847   NaN  20.785389   
2000-01-05  15.000  NaN  27.464891  ...  151.148895   NaN  21.479733   
2000-01-06  16.000  NaN  27.684412  ...  131.724564   NaN  21.370630   
2000-01-07  16.250  NaN  28.

                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...        VNO        VMC       WRB  \
Date                                ...                                   
2000-01-03  15.500  NaN  28.294199  ...   9.687466  28.422752  1.400169   
2000-01-04  14.625  NaN  27.172190  ...   9.687466  28.332516  1.357087   
2000-01-05  15.000  NaN  27.464891  ...   9.941445  30.227381  1.335547   
2000-01-06  16.000  NaN  27.684412  ...  10.068432  30.272488  1.413094   
2000-01-07

[5 rows x 489 columns]
                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...        WBA        DIS         WM  \
Date                                ...                                    
2000-01-03  15.500  NaN  28.294199  ...  20.749420  23.115221  10.629345   
2000-01-04  14.625  NaN  27.172190  ...  19.977560  24.469255  10.591113   
2000-01-05  15.000  NaN  27.464891  ...  20.204578  25.484774  10.629345   
2000-01-06  16.000  NaN  27.684412  ...  19.614326  24.46

[5 rows x 497 columns]
                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...       WDC  WU  WRK         WY  \
Date                                ...                                 
2000-01-03  15.500  NaN  28.294199  ...  3.091555 NaN  NaN  36.431850   
2000-01-04  14.625  NaN  27.172190  ...  3.680422 NaN  NaN  35.063206   
2000-01-05  15.000  NaN  27.464891  ...  3.435061 NaN  NaN  36.920658   
2000-01-06  16.000  NaN  27.684412  ...  3.582278 NaN  NaN  38.647736   

                  MMM       ABT  ABBV      ABMD  ACN      ATVI       ADBE  \
Date                                                                        
2000-01-03  28.311874  6.564564   NaN  18.25000  NaN  1.227554  16.274673   
2000-01-04  27.186905  6.377004   NaN  17.81250  NaN  1.190215  14.909401   
2000-01-05  27.974379  6.365284   NaN  18.00000  NaN  1.194882  15.204173   
2000-01-06  30.224340  6.588011   NaN  18.03125  NaN  1.171544  15.328290   
2000-01-07  30.824324  6.658342   NaN  17.93750  NaN  1.204217  16.072983   

               AMD  AAP        AES  ...  WLTW  WYNN       XEL        XRX  \
Date                                ...                                    
2000-01-03  15.500  NaN  28.294199  ...   NaN   NaN  7.782174  44.165188   
2000-01-04  14.625  NaN  27.172190  ...   NaN   NaN  7.961369  42.111004   
2000-01-05  15.000  NaN  27.464891  ...   NaN   NaN  8.268555  44.279305   
2000-01-06  16.000  NaN  27.684412  ...   NaN   NaN  8.191765  43.366337   
2000

In [ ]:
# Part 8: Correlation Tables & Data Visualization
# Visualizing data from the SP500 close price csv 
def visualize_data():
    df = pd.read_csv('sp500_joined_closes.csv')

# Example of plotting one company close price over time for Apple (ticker = AAPL) 
#    df['AAPL'].plot()
#    plt.show()

# Create correlation table for all data in df for SP500 close price
    df_corr = df.corr()
    print(df_corr.head())
# Visualize inner values of dataframe (numpy array of columns and rows)
# Specify figure and define axes using parameters(111) : one subplot is 1x1 for plot 1)
    data = df_corr.values
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)

# Define heatmap using a range from red to yellow to green
# Red = Negative, Yellow = Neutral, Green = Positive
# Colorbar for legend
    heatmap = ax.pcolor(data, cmap = plt.cm.RdYlGn)
    fig.colorbar(heatmap)
# Build graph
    ax.set_xticks(np.arange(data.shape[0]) + 0.5, minor = False)
    ax.set_yticks(np.arange(data.shape[1]) + 0.5, minor = False)
    ax.invert_yaxis()
    ax.xaxis.tick_top()

    column_labels = df_corr.columns
    row_labels = df_corr.index
# Make x labels vertical, set limit of colors (-1 = min, 1 = max)
    ax.set_xticklabels(column_labels)
    ax.set_yticklabels(row_labels)
    plt.xticks(rotation = 90)
    heatmap.set_clim(-1, 1)
    plt.tight_layout()
    plt.show()
visualize_data()

In [ ]:
# Next create features/labels, use ML for trading strategy & possible investments.	

# Part 9: Preprocessing Data for Machine Learning (ML) Model
# Part 10: Setting Target
# Part 11: Generate Labels

# svm = support-vector machine
# Cross validation to shuffle data and create training & testing samples  
# Voting classifier - use many classifiers, let them voite on best classifier

# For tutorials on Practical Machine Learning with Python visit: /
# https://pythonprogramming.net/machine-learning-tutorial-python-introduction/

# Machine Learnings: Preprocess Data for Trading Strategy

# Disclaimer: This script is for educational purposes only.
# To build more accurate ML models, you might focus on specific time frames (i.e. 1-2 yrs)
# Further, you might need more data.
# We use one-day data because it is available for free.

# Create features/labels, use ML for trading strategy & possible investments.	
# Theory: groups of companies might move up/down together (using pricing data).
# Take S&P500 dataset (closing price over time, 2000-2020), use machine learning.
# Normalize dataset by converting pricing data to % change
# Features = define, labels = target
# Labels = Buy, Sell, or Hold

In [ ]:
# Take feature data and determine label by asking:
# "Within next 7 trading days did price go up more than x% (i.e. 2%)?"
# If yes, sell company.
# If no, hold company.

# Each model generated per company
# Each company model considers pricing data from entire SP500 dataset
# To look further into the future, i.e. 30 days, change to "hm_days = 30:
def process_data_for_labels(ticker):
    hm_days = 7
    fileDataSet = pd.read_csv('sp500_joined_closes.csv', index_col = 0)
    tickers = fileDataSet.columns.values.tolist()
    fileDataSet.fillna(0, inplace = True)

# Preprocessing Data for ML

# The range will go up to a certain point (for 7 days)  
# Create custom dataset to predict future values based on percentage change
# Value in percent change = price in two days from now
# less today's price, divided by today's price, multiplied by 100.
    for i in range(1, hm_days+1):
        fileDataSet['{}d'.format(ticker, i)] = (fileDataSet[ticker].shift(-i) - fileDataSet[ticker]) / fileDataSet[ticker]

    fileDataSet.fillna(0, inplace = True)
    return tickers, fileDataSet

# Here we look at Bank of America (ticker = BAC); change ticker to view another company.
process_data_for_labels('BAC')

In [ ]:
# Next, generate labels for targets using machine learning for investing with Python
# Based on percent change pricing information, should we buy, sell, or hold company?

# See mapping functions tutorials at https://pythonprogramming.net/python-function-mapping-pandas/
# Args and kwargs tutorials at https://pythonprogramming.net/args-kwargs-intermediate-python-tutorial/

# We will also back-test out strategy.
# Example here using percent change = 2% in a week.
# If company stock price changes by 2% in 7 days get out of position for that company.
# 0 = hold, -1 = sell, +1 = buy
# If data is inbalanced you can adjust the algorithm for higher accuracy
# Aiming for > 33% accuracy (for this example)
# Example if you are incorrect on the hold, did not think it would change more than 2%, but it did...

In [ ]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

In [ ]:
# Part 11 ML Labels
# Map the function above to a column
def extract_featuresets(ticker):
    tickers, fileDataSet = process_data_for_labels(ticker)

# Define new column, value = mapped function using arg
# Arg = 7-day & change for future price; Pass paramters to function
# Generate labels for buy, sell, or hold
    fileDataSet['{}_target'.format(ticker)] = list(map(buy_sell_hold, fileDataSet[[c for c in fileDataSet.columns if c not in tickers]].values))

    vals = fileDataSet['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:', Counter(str_vals))
    
# List of options
    fileDataSet.fillna(0, inplace = True)
    fileDataSet = fileDataSet.replace([np.inf, -np.inf], np.nan)
    fileDataSet.dropna(inplace = True)

# Normalizing data set for ticker (prices) by using percent change.
    fileDataSet_vals = fileDataSet[[ticker_name for ticker_name in tickers]].pct_change()
    fileDataSet_vals = fileDataSet_vals.replace([np.inf, -np.inf], 0)
    fileDataSet_vals.fillna(0, inplace = True)

    X = fileDataSet_vals.values
    y = fileDataSet['{}_target'.format(ticker)].values

    return X, y, fileDataSet

extract_featuresets('BAC')

In [ ]:
# Create new Machine Learning function
# Create training and testing, 25% sample data will be tested against for accuracy

# Create classifier, define
# X_train is (fileDataset_vals, above) the percent change data for all companies, including company testing for
# y is the target classification (0 = hold, 1 = buy, -1 = sell)
# Use classifier that will fit input data to target 
# clf = neighbors.KNeighborsClassifier()
#
#    clf.fit(X_train, y_train)
#    confidence = clf.score(X_test, y_test)
#    print('Accuracy', confidence)

# Going forward, if you train and do not wish to retrain this model (above) pickle the classifier
# Load the pickle file, clf.predict will run and return list of predictions
# predictions = clf.predict(X_test)
# print('Predicted spread:', Counter(predictions))
#
# print('Confidence:', Counter(confidence))   
#    return confidence

# Here we are looking at Bank of America (ticker = BAC)
# do_ml('BAC')

# Example Output: 0 = hold, -1 = sell, 1 = buy (for BAC we had more holds, followed by sells and least for buys)
# Data spread: Counter({'0': 2558, '1': 1462, '-1': 1110})
# Data spread: Counter({'0': 2026, '1': 1701, '-1': 1403})
# Accuracy 0.38347622759158223
# Predicted spread: Counter({0: 758, -1: 318, 1: 207})

In [ ]:
# Part 12: ML Algorithm with Voting Classifier 
# Repeat above using Voting Classifier
def do_ml(ticker):
    X, y, fileDataSet = extract_featuresets(ticker)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    #clf = neighbors.KNeighborsClassifier()
# Replace simple classifier with voting classifier:
# Voting classifier will take list of tuples of classifier by name, classifier
# List contains tuples (i.e. 3 classifiers: linear svc, neigbors, random forest classifiers) 
    #clf = VotingClassifier([('lsvc', svm.LinearSVC()),
    clf = VotingClassifier([('lsvc', LinearSVC()), 
                            ('knn', neighbors.KNeighborsClassifier()),
                            ('rfor', RandomForestClassifier())])

    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print('Accuracy', confidence)
    predictions = clf.predict(X_test)
    print('Predicted spread:', Counter(predictions))

    return confidence

do_ml('BAC')